In [ ]:
 api_key = "RGAPI-75ae57c0-7e19-4f70-9d54-3a46dbc9ed4e"  # API 키

In [ ]:
import requests
import pandas as pd
import time
import datetime
import os
import json

In [ ]:
path = '/content/drive/MyDrive/롤 데이터 분석/meta_data'
print(f"경로 존재 여부: {os.path.exists(path)}")


경로 존재 여부: True


In [ ]:
def User_Per_Tier(tier, division):
    global cnt
    tier = tier

    # division 입력 문자로 치환
    if division == 1:
        division = 'I'
    elif division == 2:
        division = 'II'
    elif division == 3:
        division = 'III'
    elif division == 4:
        division = 'IV'
    else:
        print("잘못된 division")
        return

    globals()[tier + division] = []

    # 유저 수 카운트
    user_cnt = 0

    print(tier + ' ' + division + ' 수집 시작')

    # 조건 충족까지 반복
    while True:
        url = f'https://kr.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{tier}/{division}?page={cnt}&api_key={api_key}'
        result = requests.get(url)

        # API 상태 확인
        if result.status_code != 200:
            print(f"API 요청 실패: {result.status_code} - {result.text}")
            break

        result = result.json()

        if not isinstance(result, list):
            print(f"Unexpected response format: {result}")
            break

        if len(result) == 0:
            print("데이터 없음")
            break

        for i in range(len(result)):
            try:
                wins = int(result[i]['wins'])
                losses = int(result[i]['losses'])
                games = wins + losses
                win_ratio = (wins / games) * 100  # 승률

                # 100전 이상, 승률 50~70% 유저만 수집
                # 시즌 초기면 games 조절 가능
                if games >= 60 and 50 <= win_ratio <= 70:
                    summoner_id = result[i]['summonerId']
                    league_points = result[i]['leaguePoints']
                    rank = result[i]['rank']

                    # 필요한 데이터만 추출
                    globals()[tier + division].append({
                        'summonerId': summoner_id,
                        'tier': tier,
                        'rank': rank,
                        'leaguePoints': league_points,
                        'wins': wins,
                        'losses': losses,
                        'winRatio': win_ratio
                    })

                    user_cnt += 1
                    if user_cnt % 100 == 0:
                        print(f"{user_cnt}/100")

            except KeyError as e:
                print(f"KeyError 발생: {e}, 데이터: {result[i]}")
                continue

        if cnt % 20 == 0:
            print(f"RIOT API 카운트 횟수: {cnt}")

        if cnt % 50 == 0:
            print("RIOT API 규정으로 인해 2분간 정지합니다.")
            now = datetime.datetime.now()
            print(f"현재 시간: {now.strftime('%H시 %M분 %S초')}")
            time.sleep(70)
            print("재개합니다.")

        if len(globals()[tier + division]) >= 100:
            to_df = pd.DataFrame(globals()[tier + division])
            os.makedirs('/content/drive/MyDrive/롤 데이터 분석/meta_data', exist_ok=True)
            to_df.to_csv(f'/content/drive/MyDrive/롤 데이터 분석/meta_data/{tier}_{division}.csv', index=False, encoding='cp949')
            print(f"파일 생성 완료: {tier}_{division}.csv")
            break

        cnt += 1

    # 수집된 데이터 개수 확인
    print(f"수집된 데이터 개수: {len(globals()[tier + division])}")

# 자동화
tier_list = ['BRONZE', 'SILVER', 'GOLD', 'PLATINUM', 'EMERALD', 'DIAMOND']
division_list = [1, 2, 3, 4]

cnt = 1
for tier in tier_list:
    for division in division_list:
        User_Per_Tier(tier, division)

BRONZE I 수집 시작
100/100
파일 생성 완료: BRONZE_I.csv
수집된 데이터 개수: 109
BRONZE II 수집 시작
100/100
파일 생성 완료: BRONZE_II.csv
수집된 데이터 개수: 104
BRONZE III 수집 시작
RIOT API 카운트 횟수: 20
100/100
파일 생성 완료: BRONZE_III.csv
수집된 데이터 개수: 105
BRONZE IV 수집 시작
100/100
파일 생성 완료: BRONZE_IV.csv
수집된 데이터 개수: 107
SILVER I 수집 시작
100/100
파일 생성 완료: SILVER_I.csv
수집된 데이터 개수: 120
SILVER II 수집 시작
RIOT API 카운트 횟수: 40
100/100
파일 생성 완료: SILVER_II.csv
수집된 데이터 개수: 110
SILVER III 수집 시작
RIOT API 규정으로 인해 2분간 정지합니다.
현재 시간: 08시 01분 54초
재개합니다.
100/100
파일 생성 완료: SILVER_III.csv
수집된 데이터 개수: 110
SILVER IV 수집 시작
100/100
파일 생성 완료: SILVER_IV.csv
수집된 데이터 개수: 118
GOLD I 수집 시작
RIOT API 카운트 횟수: 60
100/100
파일 생성 완료: GOLD_I.csv
수집된 데이터 개수: 104
GOLD II 수집 시작
100/100
파일 생성 완료: GOLD_II.csv
수집된 데이터 개수: 102
GOLD III 수집 시작
100/100
파일 생성 완료: GOLD_III.csv
수집된 데이터 개수: 114
GOLD IV 수집 시작
100/100
파일 생성 완료: GOLD_IV.csv
수집된 데이터 개수: 125
PLATINUM I 수집 시작
100/100
파일 생성 완료: PLATINUM_I.csv
수집된 데이터 개수: 109
PLATINUM II 수집 시작
100/100
파일 생성 완료: PLATINUM_II.csv
수집된 데이터 개수: 121


In [ ]:
# API 호출 제한을 위한 설정

base_url = "https://asia.api.riotgames.com"
max_retries = 5  # 최대 재시도 횟수
max_calls_per_2min = 100  # 2분당 최대 API 호출 수
max_calls_per_sec = 20  # 초당 최대 API 호출 수
call_count = 0  # 현재 호출 수 카운트
start_time = time.time()  # 타이머

def get_puuid(summoner_id):
    url = f"{base_url}/lol/summoner/v4/summoners/{summoner_id}?api_key={api_key}"
    retries = 0
    while retries < max_retries:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            return data.get('puuid')
        elif response.status_code == 429:
            print(f"Rate limit exceeded for Summoner ID: {summoner_id}. Retrying after 2 minutes.")
            time.sleep(120)  # 2분 대기
        else:
            print(f"Error getting puuid for {summoner_id}: {response.status_code} - {response.text}")
            return None
        retries += 1
    print(f"Failed to get puuid for Summoner ID: {summoner_id} after {max_retries} retries.")
    return None

def add_puuid_to_csv(file_path):
    df = pd.read_csv(file_path, encoding='cp949')

    # 'puuid' 열 추가
    df['puuid'] = df['summonerId'].apply(lambda x: get_puuid(x))

    # 결과를 새로운 CSV 파일로 저장
    output_file = file_path.replace('.csv', '_with_puuid.csv')
    df.to_csv(output_file, index=False, encoding='cp949')
    print(f"파일 생성 완료: {output_file}")

def process_all_csv_files(path):
    for filename in os.listdir(path):
        if filename.endswith('.csv') and not filename.endswith('_with_puuid.csv'):
            csv_file = os.path.join(path, filename)
            print(f"처리 중: {csv_file}")
            add_puuid_to_csv(csv_file)

# 파일생성완료 메세지
process_all_csv_files('/content/drive/MyDrive/롤 데이터 분석/meta_data')

처리 중: /content/drive/MyDrive/롤 데이터 분석/meta_data/EMERALD_I.csv
Rate limit exceeded for Summoner ID: EERzb62_59wH9LfmPzGv4-EbK7gLUnkfVBqE-Hdc8SG0dUMN. Retrying after 2 minutes.
파일 생성 완료: /content/drive/MyDrive/롤 데이터 분석/meta_data/EMERALD_I_with_puuid.csv


In [ ]:
def get_match_ids(puuid):
    global call_count, start_time

    # API 호출 제한을 초과하지 않도록
    call_count += 1
    elapsed_time = time.time() - start_time

    if call_count >= max_calls_per_2min:
        if elapsed_time < 120:
            print(f"2분 동안 {max_calls_per_2min}회 호출을 완료했습니다. 2분 동안 대기합니다.")
            time.sleep(120 - elapsed_time)
        call_count = 0
        start_time = time.time()

    url = f"{base_url}/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=20&api_key={api_key}"
    retries = 0
    while retries < max_retries:
        response = requests.get(url)
        if response.status_code == 200:
            match_ids = response.json()
            return match_ids
        elif response.status_code == 429:
            print(f"Rate limit exceeded for PUUID: {puuid}. Retrying after 2 minutes.")
            time.sleep(120)  # 2분 대기 명령
        else:
            print(f"Error getting match IDs for PUUID: {puuid}: {response.status_code} - {response.text}")
            return None
        retries += 1
    print(f"Failed to get match IDs for PUUID: {puuid} after {max_retries} retries.")
    return None

def add_match_ids_to_csv(file_path):
    df = pd.read_csv(file_path, encoding='cp949')

    # matchid 열을 추가하기 위해 빈 리스트 생성
    expanded_rows = []

    for _, row in df.iterrows():
        puuid = row['puuid']
        match_ids = get_match_ids(puuid)

        if match_ids:
            for match_id in match_ids:
                new_row = row.copy()
                new_row['matchid'] = match_id
                expanded_rows.append(new_row)

        # API 요청을 제한하기 위한 대기
        if call_count % max_calls_per_sec == 0:
            time.sleep(1)

    # 데이터프레임 생성
    expanded_df = pd.DataFrame(expanded_rows)

    # 새 CSV 파일 저장
    output_file = file_path.replace('.csv', '_with_matchids.csv')
    expanded_df.to_csv(output_file, index=False, encoding='cp949')
    print(f"파일 생성 완료: {output_file}")

def process_all_csv_files(path):
    for filename in os.listdir(path):
        if filename.endswith('_with_puuid.csv') and not filename.endswith('_with_matchids.csv'):
            csv_file = os.path.join(path, filename)
            print(f"처리 중: {csv_file}")
            add_match_ids_to_csv(csv_file)

# 파일생성완료 출력
process_all_csv_files('/content/drive/MyDrive/롤 데이터 분석/meta_data')

처리 중: /content/drive/MyDrive/롤 데이터 분석/meta_data/EMERALD_I_with_puuid.csv
2분 동안 100회 호출을 완료했습니다. 2분 동안 대기합니다.
파일 생성 완료: /content/drive/MyDrive/롤 데이터 분석/meta_data/EMERALD_I_with_puuid_with_matchids.csv


In [ ]:
def get_match_metadata(match_id):
    global call_count, start_time

    # API 호출 제한을 초과하지 않도록
    call_count += 1
    elapsed_time = time.time() - start_time

    if call_count >= max_calls_per_2min:
        if elapsed_time < 120:
            print(f"2분 동안 {max_calls_per_2min}회 호출을 완료했습니다. 2분 동안 대기합니다.")
            time.sleep(120 - elapsed_time)
        call_count = 0
        start_time = time.time()

    url = f"{base_url}/lol/match/v5/matches/{match_id}?api_key={api_key}"
    retries = 0
    while retries < max_retries:
        response = requests.get(url)
        if response.status_code == 200:
            match_metadata = response.json()
            return match_metadata
        elif response.status_code == 429:
            print(f"Rate limit exceeded for match ID: {match_id}. Retrying after 2 minutes.")
            time.sleep(120)
        else:
            print(f"Error getting match metadata for match ID: {match_id}: {response.status_code} - {response.text}")
            return None
        retries += 1
    print(f"Failed to get match metadata for match ID: {match_id} after {max_retries} retries.")
    return None

def add_match_metadata_to_csv(file_path):
    df = pd.read_csv(file_path, encoding='cp949')

    expanded_rows = []

    for _, row in df.iterrows():
        match_id = row['matchid']
        match_metadata = get_match_metadata(match_id)

        if match_metadata:
            new_row = row.copy()

            # 메타데이터 전체를 JSON 형식의 문자열로 추가
            new_row['metadata_json'] = json.dumps(match_metadata)

            expanded_rows.append(new_row)

        # 요청 제한 대기
        if call_count % max_calls_per_sec == 0:
            time.sleep(1)

    expanded_df = pd.DataFrame(expanded_rows)

    # 새 CSV 파일에 저장
    output_file = file_path.replace('.csv', '_with_metadata.csv')
    expanded_df.to_csv(output_file, index=False, encoding='cp949')
    print(f"파일 생성 완료: {output_file}")

# 경로
csv_file = '/content/drive/MyDrive/롤 데이터 분석/meta_data/SILVER_IV_with_puuid_with_matchids.csv'

# 매치 메타데이터를 추가한 CSV 파일 생성 I,II,III,IV

add_match_metadata_to_csv(csv_file)

2분 동안 100회 호출을 완료했습니다. 2분 동안 대기합니다.
2분 동안 100회 호출을 완료했습니다. 2분 동안 대기합니다.
2분 동안 100회 호출을 완료했습니다. 2분 동안 대기합니다.
2분 동안 100회 호출을 완료했습니다. 2분 동안 대기합니다.
2분 동안 100회 호출을 완료했습니다. 2분 동안 대기합니다.
2분 동안 100회 호출을 완료했습니다. 2분 동안 대기합니다.
2분 동안 100회 호출을 완료했습니다. 2분 동안 대기합니다.
2분 동안 100회 호출을 완료했습니다. 2분 동안 대기합니다.
Rate limit exceeded for match ID: KR_7334757884. Retrying after 2 minutes.
Rate limit exceeded for match ID: KR_7338347110. Retrying after 2 minutes.
Rate limit exceeded for match ID: KR_7337903529. Retrying after 2 minutes.
Rate limit exceeded for match ID: KR_7338769242. Retrying after 2 minutes.
Rate limit exceeded for match ID: KR_7338547034. Retrying after 2 minutes.
Rate limit exceeded for match ID: KR_7329479595. Retrying after 2 minutes.
Rate limit exceeded for match ID: KR_7336255379. Retrying after 2 minutes.
Rate limit exceeded for match ID: KR_7338758302. Retrying after 2 minutes.
Rate limit exceeded for match ID: KR_7338075284. Retrying after 2 minutes.
Rate limit exceeded for match ID: KR_